<a href="https://colab.research.google.com/github/noheat61/my.Avatar-AI/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SetUp**

In [ ]:
# git clone
import os

!git clone https://github.com/noheat61/my.Avatar-AI avatar-ai
os.chdir(f'./avatar-ai')

In [ ]:
# 2D 모델(이미지 도메인 변환) 실행에 필요한 패키지 설치
# 이미 colab에 설치된 기존 패키지(ex) pytorch) 제외

!pip install face-alignment ninja kornia yacs

In [ ]:
# 3D 모델(3D 객체 생성) 실행에 필요한 패키지 설치
# 이미 colab에 설치된 기존 패키지(ex) pytorch) 제외
!pip install fvcore chumpy

# pytorch3d는 python, pytorch, cuda version에 따라 필요한 패키지가 다름
# 컴퓨터에 설치된 version에 맞는 패키지 설치
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

In [ ]:
# 모델 실행을 위한 대용량 파일 다운로드
# download_cartoon_data.py: CartoonStyleGAN 사용할 생성자, 인코더, 만화 생성자 등 네트워크
# download_deca_model.sh: DECA에서 사용할 템플릿 객체 파일 등

!python download_cartoon_data.py
!bash download_deca_model.sh

# **Run with Your Own Image**

In [ ]:
# colab에 이미지 파일 업로드(/content/avatar-ai에 저장)

from google.colab import files

uploaded = files.upload()
image_name = list(uploaded.keys())[0]
print(image_name)

In [ ]:
# 2D, 3D 모델 불러오기(infer.py)
# 도메인 변환한 이미지, 생성한 3D 아바타를 저장할 폴더 지정

from infer import Model2D, Model3D

model2D = Model2D()
model3D = Model3D()

cartoon_path = "cartoon_image/"
avatar_path = "avatar/"

In [ ]:
# 폴더가 없으면 인퍼런스에서 오류 발생
# 미리 해당 폴더 만들기

import os

if not os.path.exists("cartoon_image"):
    os.mkdir("cartoon_image")
if not os.path.exists("avatar"):
    os.mkdir("avatar")

In [ ]:
# 2D 모델(이미지 도메인 변환) 인퍼런스
# image_name을 입력 이미지 경로로 입력받아 출력 폴더에 만화화된 이미지들 저장
# make_all -> True : 생성할 수 있는 모든 스타일의 이미지 생성
# make_all -> False : style에 입력된 스타일(기본: 디즈니)의 이미지만 생성

model2D.inference(input_path=image_name, output_path=cartoon_path, make_all=True, style="DISNEY") # style: DISNEY, COMICS, ART, 여신강림, 외모지상주의

In [ ]:
# 3D 모델(3D 객체 생성) 인퍼런스
# 입력 폴더에 있는 이미지들(만화화된 이미지들)을 모두 3D 객체로 변환하여 아바타 폴더에 저장
# get_full -> False : 머리 부분을 제외한 얼굴 전면부만 텍스처 입힘
# get_full -> True : 색칠되지 않은 다른 부분에 대해 예측하여 모든 얼굴에 텍스처 입힘(자연스럽지 않음)

model3D.inference(input_path=cartoon_path, output_path=avatar_path, get_full = False)

# **Download results**

In [ ]:
# 만들어진 객체 다운로드
# avatar/ 폴더에 있는 객체들을 zip파일로 묶어 다운로드

from google.colab import files

!zip -r avatar_results.zip avatar
files.download("avatar_results.zip")